In [1]:
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import torch

import pandas as pd
import numpy as np
import math

import nltk

from nltk.tokenize import word_tokenize, wordpunct_tokenize


%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
# import datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 3.4 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.26.15 which is incompatible.
cloud-ml 0.0.1 requires requests<=2.25.1,>=2.22.0, but you have requests 2.28.2 which is incompatible.
cloud-ml 0.0.1 requires tqdm<=4.54.1,>=4.45.0, but you have tqdm 4.65.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [4]:
model_name = "ai-forever/ruRoberta-large"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [11]:
DATA_PATH = '/home/jupyter/mnt/s3/dataset_hackaton/'

In [12]:
!ls /home/jupyter/mnt/s3/dataset_hackaton/

bank-categories.csv
bank-sentiment.csv
banks-ethic-DB - NEW.docx
homework_shad_da.ipynb.txt
sentiment.csv
train.csv


In [44]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')

bank_categories_df = pd.read_csv(DATA_PATH + 'bank-sentiment.csv')
bank_sentiment_df = pd.read_csv(DATA_PATH + 'bank-categories.csv')


In [47]:
train_df.head()

,Unnamed: 0,sentence,1category,2category,sentiment
0,4754,При этом всегда получал качественные услуги.,Communication,NaN,+
1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,NaN,−
2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,NaN,−
3,11640,Отвратительное отношение к клиентам.,Communication,NaN,−
4,5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,NaN,+


In [48]:
# indexes = np.arange(train_df.shape[0])
# np.random.shuffle(indexes)
# train_size = math.ceil(train_df.shape[0] * 0.8)

# train_indexes = indexes[:train_size]
# test_indexes = indexes[train_size:]

# train_data = train_df.iloc[train_indexes]
# test_data = train_df.iloc[test_indexes]

# train_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)


indexes = np.arange(train_df.shape[0])
np.random.shuffle(indexes)
train_size = math.ceil(train_df.shape[0] * 0.8)

data = train_df.iloc[indexes]

In [49]:
from torch.utils.data import Dataset, random_split


class Dataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, tokenizer):
        self.tokenizer = tokenizer
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column
        self.label2num = lambda x: x == '+'

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)

        return {"tokens": tokens, "target": label}

    def get_tokens_(self, text):
          return self.tokenizer(text)

    def __len__(self):
        return self.data.shape[0]

In [5]:
dataset = Dataset(data, 'sentence', 'sentiment', tokenizer)

NameError: name 'Dataset' is not defined

In [51]:
dataset

In [61]:
from torch.utils.data import DataLoader


batch_size = 128
num_workers = 2

def average_emb(batch):
    features = [b["tokens"] for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": features, "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dataset) * 0.8)

# train, valid = random_split(dataset, [train_size, len(train_data) - train_size])
train, valid = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [62]:
next(iter(train_loader))

{'features': [['За',
   'это',
   'первый',
   'минус',
   '.',
   '2',
   ')',
   '29',
   'ноября',
   'я',
   'обратился',
   'лично',
   'в',
   'отделение',
   'банка',
   ':',
   '4',
   '"',
   'Московский',
   '"',
   '196066',
   ',',
   'Город',
   'Санкт',
   '-',
   'Петербург',
   ',',
   'Ленинский',
   'проспект',
   '161',
   ',',
   'пом',
   '.',
   '22',
   '-',
   'Н',
   ',',
   'литера',
   'А',
   '-',
   'для',
   'открытия',
   'карты',
   '.'],
  ['Сразу',
   'могу',
   'сказать',
   ',',
   'что',
   'картой',
   'пользуюсь',
   'почти',
   '2',
   'года',
   'и',
   'всем',
   'доволен',
   '.'],
  ['Прошу',
   'разобраться',
   'в',
   'этой',
   'проблеме',
   ',',
   'потому',
   'что',
   'навязанная',
   'услуга',
   '-',
   'это',
   'прямое',
   'нарушение',
   'закона',
   'о',
   'защите',
   'прав',
   'потребителей',
   '.'],
  ['За',
   'последние',
   'годы',
   'я',
   'пользовался',
   'услугами',
   'разных',
   'банков',
   ',',
   'но',
   